# Chessformer - Data analysis

## Imports

In [2]:
import duckdb

## Data analysis

In [8]:
duckdb.query('''
SELECT *
FROM read_parquet('data/raw_game_data/*.parquet.zstd')
LIMIT 10
''').df()

,Timestamp,Result,WhiteElo,BlackElo,TimeControl,Termination,Moves
0,2016-11-30 21:00:00-02:00,0-1,1178,1520,360+0,Normal,p e2 e4 - p d7 d5 - p d2 d3 - p d5 d4 - n g1 f...
1,2016-11-30 21:00:02-02:00,1-0,1707,1624,300+0,Normal,p d2 d4 - p d7 d5 - p c2 c4 - p d5 c4 - n b1 c...
2,2016-11-30 21:00:01-02:00,1-0,1626,1750,300+0,Normal,p e2 e4 - p d7 d5 - p e4 e5 - p d5 d4 - n g1 f...
3,2016-11-30 21:00:04-02:00,0-1,1527,1214,300+0,Normal,p e2 e4 - p d7 d5 - p d2 d4 - p d5 e4 - n b1 c...
4,2016-11-30 21:00:05-02:00,1/2-1/2,1997,1956,180+0,Normal,p d2 d4 - n g8 f6 - b c1 f4 - p d7 d5 - p e2 e...
5,2016-11-30 21:00:07-02:00,0-1,1686,1623,300+3,Time forfeit,p d2 d4 - p d7 d5 - p c2 c4 - n b8 c6 - n g1 f...
6,2016-11-30 21:00:07-02:00,1-0,1741,1876,120+2,Time forfeit,p d2 d4 - p f7 f5 - p c2 c4 - p g7 g6 - n b1 c...
7,2016-11-30 21:00:07-02:00,0-1,1726,1741,300+0,Normal,p e2 e4 - p e7 e5 - n g1 f3 - n b8 c6 - b f1 e...
8,2016-11-30 21:00:07-02:00,0-1,1743,1749,300+1,Normal,p d2 d4 - n g8 f6 - p c2 c4 - p e7 e6 - n g1 f...
9,2016-11-30 21:00:08-02:00,1-0,1852,1956,180+0,Time forfeit,p g2 g3 - p d7 d5 - b f1 g2 - n g8 f6 - p d2 d...


In [10]:
duckdb.query('''
SELECT
    COUNT(*) AS num_games,
    MIN(LEN(Moves) -  LEN(REPLACE(Moves, ' ', '')) + 1) AS min_tokens,
    MAX(LEN(Moves) -  LEN(REPLACE(Moves, ' ', '')) + 1) AS max_tokens,
FROM read_parquet('data/raw_game_data/*.parquet.zstd')
WHERE
    Termination = 'Normal'
    AND LEN(Moves) -  LEN(REPLACE(Moves, ' ', '')) + 1 <= 512
    AND TimeControl = '180+0'
''').df()

,num_games,min_tokens,max_tokens
0,40301488,4,512
